In [1]:
import sys
sys.path.append('..')

In [14]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from matplotlib import pyplot as plt
from matplotlib import image as img

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

from config import load_train_data

In [3]:
x, y = load_train_data(rescaling_factor=3)

x.shape, y.shape

100%|██████████| 1000/1000 [00:12<00:00, 78.03it/s]


((1000, 160, 214, 3), (1000, 7))

In [16]:
y_scaled = MinMaxScaler().fit_transform(y)

In [4]:
def get_random_crop(image: np.ndarray, cropped_size: int) -> np.ndarray:
    """Get a random square crop of the given image.
    """
    
    half_size = cropped_size // 2

    height, width, _ = image.shape
    center_x = np.random.randint(half_size, height - half_size)
    center_y = np.random.randint(half_size, width - half_size)
    
    return image[center_x - half_size : center_x + half_size, center_y - half_size : center_y + half_size]

In [5]:
def get_center_crop(image: np.ndarray, cropped_size: int) -> np.ndarray:
    """Get the center crop of the given image.
    """

    half_size = cropped_size // 2
    
    height, width, _ = image.shape
    center_x = height // 2
    center_y = width // 2
    
    return image[center_x - half_size : center_x + half_size, center_y - half_size : center_y + half_size]

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y_scaled, test_size=0.25, shuffle=True)

In [18]:
AUGMENTING_RATE = 30
CROPPED_SIZE = 80

In [19]:
x_train_augmented = []
y_train_augmented = []

for x_, y_ in zip(x_train, y_train):
    for _ in range(AUGMENTING_RATE):
        x_train_augmented.append(get_random_crop(x_, cropped_size=CROPPED_SIZE))
        y_train_augmented.append(y_)

x_train_augmented = np.array(x_train_augmented)
y_train_augmented = np.array(y_train_augmented)

x_train_augmented.shape, y_train_augmented.shape

((22500, 80, 80, 3), (22500, 7))

In [20]:
x_test_centered = np.array([get_center_crop(image, cropped_size=CROPPED_SIZE) for image in x_test])

x_test_centered.shape, y_test.shape

((250, 80, 80, 3), (250, 7))

In [30]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=x_train_augmented[0].shape),
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(y_train_augmented.shape[1]),
])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 78, 78, 16)        448       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 39, 39, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 37, 37, 32)        4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 18, 18, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 4096)             

In [31]:
EPOCHS = 15

In [32]:
model.compile(loss='mse')

In [33]:
model.fit(
    x_train_augmented, y_train_augmented,
    epochs=EPOCHS,
    validation_data=(x_test_centered, y_test),
)

Epoch 1/15
704/704 [==============================] - 16s 22ms/step - loss: 0.0558 - val_loss: 0.0312
Epoch 2/15
704/704 [==============================] - 16s 23ms/step - loss: 0.0239 - val_loss: 0.0158
Epoch 3/15
704/704 [==============================] - 16s 22ms/step - loss: 0.0169 - val_loss: 0.0163
Epoch 4/15
704/704 [==============================] - 16s 23ms/step - loss: 0.0134 - val_loss: 0.0141
Epoch 5/15
704/704 [==============================] - 15s 21ms/step - loss: 0.0111 - val_loss: 0.0208
Epoch 6/15
704/704 [==============================] - 15s 22ms/step - loss: 0.0096 - val_loss: 0.0139
Epoch 7/15
704/704 [==============================] - 15s 21ms/step - loss: 0.0082 - val_loss: 0.0122
Epoch 8/15
704/704 [==============================] - 16s 23ms/step - loss: 0.0072 - val_loss: 0.0133
Epoch 9/15
704/704 [==============================] - 17s 23ms/step - loss: 0.0065 - val_loss: 0.0147
Epoch 10/15
704/704 [==============================] - 16s 22ms/step - loss: 0.005

In [34]:
def posenet_loss(y_true, y_pred, beta: float = 300):
    """Squared error sum with promoting the quaternion part error by the `beta` argument.
    """

    y_true_pos = y_true[:, :4]
    y_true_q = y_true[:, 4:]
    y_pred_pos = y_pred[:, :4]
    y_pred_q = y_pred[:, 4:]

    return (tf.reduce_sum(tf.square(y_true_pos - y_pred_pos), axis=1)
            + beta * tf.reduce_sum(tf.square(y_true_q - y_pred_q), axis=1))

In [35]:
model.compile(loss=posenet_loss, metrics=['mse'])

In [36]:
model.fit(
    x_train_augmented, y_train_augmented,
    epochs=EPOCHS,
    validation_data=(x_test_centered, y_test),
)

Epoch 1/15
704/704 [==============================] - 16s 21ms/step - loss: 3.9786 - mse: 0.0051 - val_loss: 11.2790 - val_mse: 0.0129
Epoch 2/15
704/704 [==============================] - 15s 21ms/step - loss: 3.5558 - mse: 0.0057 - val_loss: 7.8060 - val_mse: 0.0100
Epoch 3/15
704/704 [==============================] - 15s 21ms/step - loss: 3.4598 - mse: 0.0059 - val_loss: 10.5906 - val_mse: 0.0125
Epoch 4/15
704/704 [==============================] - 15s 21ms/step - loss: 3.0968 - mse: 0.0061 - val_loss: 12.9805 - val_mse: 0.0135
Epoch 5/15
704/704 [==============================] - 15s 21ms/step - loss: 2.8064 - mse: 0.0061 - val_loss: 15.9404 - val_mse: 0.0220
Epoch 6/15
704/704 [==============================] - 15s 21ms/step - loss: 2.8139 - mse: 0.0066 - val_loss: 10.2331 - val_mse: 0.0135
Epoch 7/15
704/704 [==============================] - 15s 21ms/step - loss: 2.4901 - mse: 0.0062 - val_loss: 10.9041 - val_mse: 0.0129
Epoch 8/15
704/704 [==============================] - 15